In [11]:
import osmnx as ox
import geopandas as gpd
from shapely.ops import cascaded_union
import pandas as pd

In [2]:
borough_bds = gpd.read_file('borough_bds.geojson')
## Source: https://stackoverflow.com/questions/40385782/
## make-a-union-of-polygons-in-python-geopandas-or-shapely-into-a-single-geometr
polygon = gpd.GeoSeries(cascaded_union(borough_bds['geometry']))
polygon

0    (POLYGON ((-74.05050806403247 40.5664220341608...
dtype: object

In [3]:
G = ox.graph_from_polygon(polygon[0], network_type='drive')

In [ ]:
G_projected = ox.project_graph(G)
fig, ax = ox.plot_graph(G_projected)

In [5]:
nodes_proj, edges_proj = ox.graph_to_gdfs(G, nodes=True, edges=True)
edges_proj.head()

,access,bridge,geometry,highway,junction,key,lanes,length,maxspeed,name,oneway,osmid,ref,service,tunnel,u,v,width
0,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.939496...",residential,NaN,0,NaN,81.315,20 mph,47th Avenue,False,5704407,NaN,NaN,NaN,42860546,42817478,NaN
1,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.941638...",residential,NaN,0,NaN,114.648,20 mph,47th Avenue,False,5704407,NaN,NaN,NaN,42860546,42860543,NaN
2,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.940583...",residential,NaN,0,NaN,161.961,NaN,27th Street,False,5705160,NaN,NaN,NaN,42860546,42872051,NaN
3,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.940428...",residential,NaN,0,NaN,87.217,NaN,27th Street,False,5705160,NaN,NaN,NaN,42860546,42872047,NaN
4,NaN,NaN,"LINESTRING (-73.93367600000001 40.655156, -73....",residential,NaN,0,NaN,170.680,NaN,Schenectady Avenue,False,5679907,NaN,NaN,NaN,42467330,42510401,NaN


In [6]:
speed_lims = gpd.read_file('speed_lims.geojson')
speed_lims.drop(['shape_leng', 'postvz_sg', 'street'], axis=1, inplace=True)
speed_lims['postvz_sl'][speed_lims['postvz_sl'] == '0'] = 25

In [ ]:
speed_lims_bounds = speed_lims['geometry'].bounds
speed_lims['u'] = ox.get_nearest_nodes(G, speed_lims_bounds['minx'],\
    speed_lims_bounds['miny'], method='kdtree')
speed_lims['v'] = ox.get_nearest_nodes(G, speed_lims_bounds['maxx'],\
    speed_lims_bounds['maxy'], method='kdtree')
speed_lims['A'] = speed_lims['u'].map(str) + ' ' + speed_lims['v'].astype(str)
speed_lims['B'] = speed_lims['v'].map(str) + ' ' + speed_lims['u'].astype(str)

In [8]:
speed_lims.drop(['geometry', 'u', 'v'], axis=1, inplace=True)

In [12]:
edges_proj['A'] = edges_proj['u'].map(str) + ' ' + edges_proj['v'].astype(str)

### merge the data frame    
merged_df_1 = edges_proj.merge(speed_lims[['postvz_sl', 'A']], how='left', on=['A'])
merged_df_2 = edges_proj.merge(speed_lims[['postvz_sl', 'B']], how='left', left_on=['A'], right_on=['B'])

merged_df = merged_df_1.merge(merged_df_2[['postvz_sl', 'A']], on=['A'])
merged_df.drop('A', axis=1, inplace=True)

nan_speed = pd.isnull(merged_df['postvz_sl_x'])
merged_df['postvz_sl_x'][nan_speed] = merged_df['postvz_sl_y'][nan_speed]
merged_df.drop('postvz_sl_y', axis=1, inplace=True)
nan_speed_2 = pd.isnull(merged_df['postvz_sl_x'])
merged_df['postvz_sl_x'][nan_speed_2] = 25

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
merged_df.head()

,access,bridge,geometry,highway,junction,key,lanes,length,maxspeed,name,oneway,osmid,ref,service,tunnel,u,v,width,postvz_sl_x
0,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.939496...",residential,NaN,0,NaN,81.315,20 mph,47th Avenue,False,5704407,NaN,NaN,NaN,42860546,42817478,NaN,25
1,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.941638...",residential,NaN,0,NaN,114.648,20 mph,47th Avenue,False,5704407,NaN,NaN,NaN,42860546,42860543,NaN,25
2,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.940583...",residential,NaN,0,NaN,161.961,NaN,27th Street,False,5705160,NaN,NaN,NaN,42860546,42872051,NaN,25
3,NaN,NaN,"LINESTRING (-73.9404517 40.7436274, -73.940428...",residential,NaN,0,NaN,87.217,NaN,27th Street,False,5705160,NaN,NaN,NaN,42860546,42872047,NaN,25
4,NaN,NaN,"LINESTRING (-73.93367600000001 40.655156, -73....",residential,NaN,0,NaN,170.680,NaN,Schenectady Avenue,False,5679907,NaN,NaN,NaN,42467330,42510401,NaN,25
